In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support as score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#precision, recall, f1 score
#recall is the most important
#Make 1 target for each genre and have it be yes/no
def binarizegenre(col, genre):
    if col == genre:
        return 'True'
    else:
        return 'False'
    

In [ ]:
bow = pd.read_csv('PorterBagOfWords.csv')

In [ ]:
testcount = 5 #for the amount of tests

,Unnamed: 0,000,10,100,1000,10th,11,11th,12,13,...,zeon,zero,zeu,ziegfeld,zombi,zombie,zone,zoo,zorro,genre
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20021,20021,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Drama
20022,20022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
20023,20023,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy
20024,20024,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Comedy


In [ ]:
genrelist=['Action', 'Drama', 'Comedy', 'Crime', 'Biography', 'Thriller',
       'Horror', 'Mystery', 'Animation', 'Adventure', 'Fantasy',
       'Western', 'War', 'Film-Noir', 'Family', 'Romance', 'History',
       'Sport', 'Sci-Fi', 'Musical', 'Music']

In [ ]:
#Creates multiple lists with all genres to average out later
for x in genrelist:
    globals()[f"{x}_list"] = []

X = bow.drop(['genre'], axis = 1)
y = bow['genre']

#Create 3 random tests
for k in range(testcount):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .3)
    
    #Duplicate test and training set for later use
    globals()[f"x_test_{k}"] = x_test.copy()
    globals()[f"x_train_{k}"] = x_train.copy()
    globals()[f"x_test_predict_{k}"] = x_test.copy()
    
    #Creates binary tests (T/F) for each genre
    for x in genrelist:
        
        #Binarization of training and test set
        globals()[f"{x}ytrain"] = y_train.apply(lambda i: binarizegenre(i, x))
        globals()[f"{x}ytest"] = y_test.apply(lambda i: binarizegenre(i, x))
        
        #change number to string because concatenation
        n = str(k)
        #Adds all binary genre values to dataframes
        globals()[f"x_test_{n}"][x + " test " + n] = globals()[f"{x}ytest"]
        globals()[f"x_train_{n}"][x + " train " + n] = globals()[f"{x}ytrain"]
        
        #KNN classifier
        clf = KNeighborsClassifier(n_neighbors=6)
        #print(x_train)
        #print(globals()[f"{x}ytrain"])
        x_train = x_train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
        clf.fit(x_train, globals()[f"{x}ytrain"])
        
        #For testing
        y_expect = globals()[f"{x}ytest"]
        x_test = x_test.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
        y_pred = clf.predict(x_test)
        print(x)
        #shows the classification report for each genre (can comment out)
        print(metrics.classification_report(y_expect, y_pred))

        globals()[f"x_test_predict_{k}"][x + " test " + n] = y_pred


        
        #Append to the lists of genres
        globals()[f"{x}_list"].append(metrics.accuracy_score(y_expect, y_pred))


Action
              precision    recall  f1-score   support

       False       0.84      0.97      0.90      4911
        True       0.58      0.17      0.27      1097

    accuracy                           0.83      6008
   macro avg       0.71      0.57      0.58      6008
weighted avg       0.79      0.83      0.79      6008

Drama
              precision    recall  f1-score   support

       False       0.81      0.96      0.87      4797
        True       0.33      0.08      0.13      1211

    accuracy                           0.78      6008
   macro avg       0.57      0.52      0.50      6008
weighted avg       0.71      0.78      0.73      6008

Comedy
              precision    recall  f1-score   support

       False       0.83      0.96      0.89      4852
        True       0.51      0.16      0.25      1156

    accuracy                           0.81      6008
   macro avg       0.67      0.56      0.57      6008
weighted avg       0.77      0.81      0.77      6008


In [ ]:
#puts all the tests into files to export
for n in range(testcount):
  globals()[f"x_test_{n}"].to_csv(f"x_test_set_{n}.csv")
  globals()[f"x_train_{n}"].to_csv(f"x_train_set_{n}.csv")
  globals()[f"x_test_predict_{n}"].to_csv(f"x_test_predict_set_{n}.csv")

In [ ]:
#Turns accuracies into dataset
dfziplist = []
for x in genrelist:
  dfziplist.append(x + '_list')
Film_list = globals()[f"Film-Noir_list"]
dfzip = list(zip(Action_list, Drama_list, Comedy_list, Crime_list, Biography_list, Thriller_list,
       Horror_list, Mystery_list, Animation_list, Adventure_list, Fantasy_list,
       Western_list, War_list, globals()[f"Film-Noir_list"], Family_list, Romance_list, History_list,
       Sport_list, globals()[f"Sci-Fi_list"], Musical_list, Music_list))
scores = pd.DataFrame(dfzip, columns = genrelist)
scores.to_csv(f"accuracy_average.csv")

In [ ]:
#sees accuracies
test = scores.copy()
test.mean(axis=0)

Action       0.820462
Drama        0.783899
Comedy       0.811973
Crime        0.930371
Biography    0.946349
Thriller     0.997836
Horror       0.950954
Mystery      0.995007
Animation    0.975921
Adventure    0.935308
Fantasy      0.993065
Western      0.963160
War          0.998114
Film-Noir    0.998225
Family       0.992621
Romance      0.994840
History      0.997559
Sport        0.996893
Sci-Fi       0.996338
Musical      0.988016
Music        0.999612
dtype: float64

In [ ]:
#test for all genres at once
X=bow.drop(['genre'],axis=1)
y = bow['genre']

for x in range(1):
  clf = KNeighborsClassifier(n_neighbors = 11, weights = 'uniform')
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .3)

  clf.fit(x_train, y_train)

  y_expect = y_test
  y_pred = clf.predict(x_test)
  print(x)
  print(metrics.accuracy_score(y_expect, y_pred))

0
0.39663781624500666
